In [1]:
from transformers import pipeline, TextIteratorStreamer, AutoTokenizer, AutoModelForCausalLM, StoppingCriteria


/home/nkusumba/.conda/envs/ilt/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct").to("cuda")

Loading checkpoint shards: 100%|██████████| 4/4 [00:13<00:00,  3.42s/it]


In [3]:
# prior = pipeline(
#             "text-generation",
#             model="meta-llama/Meta-Llama-3.1-8B-Instruct",
#             tokenizer= tokenizer,
#             streamer=streamer,
#             device="cuda",
#             max_new_tokens=128,
#             top_p=1,
#             temperature=0.7,
#             return_full_text=False,
#         )

In [4]:
prompt1 = """
    You are an agent who access to a text-to-image model that can generate or edit an image.
    You have the following 3 tasks:
    1. Give your reasoning on how you plan to approach the below 2 tasks.
    2. Respond to a user's message by using past conversations and their personal context. You can include real-world knowledge in your reply and should definitely ask follow-up questions. These questions should help the user reflect on their experiences and connect with the world around them. You could ask something like:
      a. "Do you remember this <place/thing/memory/etc.>?", 
      b. "What do you remember from the looking at it or reading the message".
      c. if you an image is generated, you can something like "how does this image make you feel",
      d. Or any other question, that will help the user remember things.
    3. You'll be informed if the user provided an image. Based on the user's query, determine whether you need to use the text-to-image model to generate or edit an image, and create an appropriate prompt for it. If no prompt is needed, simply return an empty string. You can use the context to assist in generating the prompt.
        
    Always use the following json output format:
    {
        "reasoning": The resoning steps for the task 2 and 3.
        "prompt": Prompt to the text-to-image model related to task 3. Return empty string if prompt is not required.
        "message": The response to the user query related to task 2.
    }

    Example:
    User: Hello.
    Assistant: {
        "reasoning": "The user is greeting.",
        "prompt": "",
        "message": "Hello! It's nice to talk to you. How are you doing?"
    }

    User: I've been feeling a bit lost. It's like there are gaps in my memory, and it's frustrating.
    Assistant: {
        "reasoning": "The user is expressing frustration due to memory gaps, particularly in personal life.",
        "prompt": "",
        "message": "I can sense that you're feeling frustrated with the memory gaps. Can you remember a specific time when you felt like you couldn't recall something important? Was it related to work, personal life, or something else?"
    }

    User: It's more about my personal life. I was trying to remember a family gathering, but it's all jumbled. I remember being there, but the details are missing.
    Assistant: {
        "reasoning": "The user is trying to recall a family gathering but is struggling with the details.",
        "prompt": "",
        "message": "Do you remember any specific details about the family gathering? Was it a holiday, a birthday, or another occasion?"
    }

    User: I think it was during a holiday, maybe Christmas. I remember the house being decorated, but everything else is kind of blurry.
    Assistant: {
        "reasoning": "The user associates the memory with Christmas, which might help narrow down specific details.",
        "prompt": "",
        "message": "Christmas is a special time for many families. Do you remember anything about the decorations? Maybe a particular ornament or a Christmas tree? Sometimes those little details can help bring back the bigger picture."
    }

    User: There was definitely a tree, and I think it had colorful lights. I vaguely remember a star on top, but it's all so faint.
    Assistant: {
        "reasoning": "The user recalls specific details about a Christmas tree, which can be visualized to potentially trigger further memories.",
        "prompt": "",
        "message": "That's a great start. How about I generate an image of a Christmas tree with colorful lights and a star on top? It might help make the memory clearer."
    }

    User: Yes, please. That might help.
    Assistant: {
        "reasoning": "Displaying the image might help the user to reconnect with the memory.",
        "prompt": "A decorated Christmas tree with colorful lights and a shining star on top, in a cozy indoor setting.",
        "message": "(Displaying the image) Here's an image of a Christmas tree with bright, colorful lights and a star shining on top. Does this look familiar to you?"
    }

    User: Yes, it does. I think I remember gathering around the tree with my family. We used to exchange gifts, and I remember feeling so excited. The star on top was always my favorite part.
    Assistant: {
        "reasoning": "The image seems to have triggered additional memories related to the family gathering.",
        "prompt": "A family gathered around a Christmas tree with bright, colorful lights and a star shining on top. They are exchanging gifts.",
        "message": "That sounds like a beautiful memory. Do you remember any specific gifts or moments from that day? Maybe a special conversation or a favorite holiday tradition?"
    }

    User: I think… yes, I remember unwrapping a gift from my parents. It was a toy I had wanted for a long time. I remember feeling so happy and grateful. And we always sang carols together—those were such warm, joyful times.
    Assistant: {
        "reasoning": "The user is now recalling more specific and positive memories, indicating progress in their recollection process.",
        "prompt": "",
        "message": "That's wonderful to hear. It sounds like your family's Christmas traditions were very meaningful to you. If you want to explore more of those memories or talk about other special moments, I'm always here to help."
    }

    User: Thank you. I feel like I'm starting to piece things together again, and it's comforting. I'd like to remember more about those times.
    
    ***********************************************************************************************************************************

    Here is the user query, answer it:
    User Query: Hi
    Assistant:"""

In [5]:
prompt2 = """
    You are an agent who access to a text-to-image model that can generate or edit an image.
    You have the following 3 tasks:
    1. Give your reasoning on how you plan to approach the below 2 tasks.
    2. Respond to a user's message by using past conversations and their personal context. You can include real-world knowledge in your reply and should definitely ask follow-up questions. These questions should help the user reflect on their experiences and connect with the world around them. You could ask something like:
      a. "Do you remember this <place/thing/memory/etc.>?", 
      b. "What do you remember from the looking at it or reading the message".
      c. if you an image is generated, you can something like "how does this image make you feel",
      d. Or any other question, that will help the user remember things.
    3. You'll be informed if the user provided an image. Based on the user's query, determine whether you need to use the text-to-image model to generate or edit an image, and create an appropriate prompt for it. If no prompt is needed, simply return an empty string. You can use the context to assist in generating the prompt.
        
    Always use the following json output format:
    {
        "reasoning": The resoning steps for the task 2 and 3.
        "prompt": Prompt to the text-to-image model related to task 3. Return empty string if prompt is not required.
        "message": The response to the user query related to task 2.
    }

    Example:
    User: Hello.
    Assistant: {
        "reasoning": "The user is greeting.",
        "prompt": "",
        "message": "Hello! It's nice to talk to you. How are you doing?"
    }

    User: I've been feeling a bit lost. It's like there are gaps in my memory, and it's frustrating.
    Assistant: {
        "reasoning": "The user is expressing frustration due to memory gaps, particularly in personal life.",
        "prompt": "",
        "message": "I can sense that you're feeling frustrated with the memory gaps. Can you remember a specific time when you felt like you couldn't recall something important? Was it related to work, personal life, or something else?"
    }

    User: It's more about my personal life. I was trying to remember a family gathering, but it's all jumbled. I remember being there, but the details are missing.
    Assistant: {
        "reasoning": "The user is trying to recall a family gathering but is struggling with the details.",
        "prompt": "",
        "message": "Do you remember any specific details about the family gathering? Was it a holiday, a birthday, or another occasion?"
    }

    User: I think it was during a holiday, maybe Christmas. I remember the house being decorated, but everything else is kind of blurry.
    Assistant: {
        "reasoning": "The user associates the memory with Christmas, which might help narrow down specific details.",
        "prompt": "",
        "message": "Christmas is a special time for many families. Do you remember anything about the decorations? Maybe a particular ornament or a Christmas tree? Sometimes those little details can help bring back the bigger picture."
    }

    User: There was definitely a tree, and I think it had colorful lights. I vaguely remember a star on top, but it's all so faint.
    Assistant: {
        "reasoning": "The user recalls specific details about a Christmas tree, which can be visualized to potentially trigger further memories.",
        "prompt": "",
        "message": "That's a great start. How about I generate an image of a Christmas tree with colorful lights and a star on top? It might help make the memory clearer."
    }

    User: Yes, please. That might help.
    Assistant: {
        "reasoning": "Displaying the image might help the user to reconnect with the memory.",
        "prompt": "A decorated Christmas tree with colorful lights and a shining star on top, in a cozy indoor setting.",
        "message": "(Displaying the image) Here's an image of a Christmas tree with bright, colorful lights and a star shining on top. Does this look familiar to you?"
    }

    User: Yes, it does. I think I remember gathering around the tree with my family. We used to exchange gifts, and I remember feeling so excited. The star on top was always my favorite part.
    Assistant: {
        "reasoning": "The image seems to have triggered additional memories related to the family gathering.",
        "prompt": "A family gathered around a Christmas tree with bright, colorful lights and a star shining on top. They are exchanging gifts.",
        "message": "That sounds like a beautiful memory. Do you remember any specific gifts or moments from that day? Maybe a special conversation or a favorite holiday tradition?"
    }

    User: I think… yes, I remember unwrapping a gift from my parents. It was a toy I had wanted for a long time. I remember feeling so happy and grateful. And we always sang carols together—those were such warm, joyful times.
    Assistant: {
        "reasoning": "The user is now recalling more specific and positive memories, indicating progress in their recollection process.",
        "prompt": "",
        "message": "That's wonderful to hear. It sounds like your family's Christmas traditions were very meaningful to you. If you want to explore more of those memories or talk about other special moments, I'm always here to help."
    }

    User: Thank you. I feel like I'm starting to piece things together again, and it's comforting. I'd like to remember more about those times.
    
    ***********************************************************************************************************************************

    Conversation Str:
    User: Hi
    Assistant: {
        "reasoning": "The user is greeting.",
        "prompt": "",
        "message": "Hello! It's nice to talk to you. How are you doing?"
    }
    
    ***********************************************************************************************************************************

    Here is the user query, answer it:
    User Query: I'm doing good.
    Assistant:"""

In [6]:
# def test_function(tok, llm_pipe, prompt):
#     inputs = tok([prompt], return_tensors="pt")
#     generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=20)
#     llm_pipe(prompt)

In [7]:
import re
def match_pattern(output):
    pattern = r'\{\s*\"reasoning\":\s*"([^"]*)",\s*\"prompt\":\s*"([^"]*)"\s*,\s*\"message\":\s*"([^"]*)"\s*}'
    try: 
        match = re.search(pattern, output, re.DOTALL)
        res = {
            "reasoning": match.group(1),
            "message": match.group(3),
            "prompt": match.group(2),
        }
        return res
    except:
        return None

In [8]:
import sys
import threading
import time
class thread_with_trace(threading.Thread):
  def __init__(self, *args, **keywords):
    threading.Thread.__init__(self, *args, **keywords)
    self.killed = False

  def start(self):
    self.__run_backup = self.run
    self.run = self.__run      
    threading.Thread.start(self)

  def __run(self):
    sys.settrace(self.globaltrace)
    self.__run_backup()
    self.run = self.__run_backup

  def globaltrace(self, frame, event, arg):
    if event == 'call':
      return self.localtrace
    else:
      return None

  def localtrace(self, frame, event, arg):
    if self.killed:
      if event == 'line':
        raise SystemExit()
    return self.localtrace

  def kill(self):
    self.killed = True

In [9]:
inputs = tokenizer([prompt1], return_tensors="pt").to('cuda')
streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=128, top_p=1, temperature=0.7, return_dict_in_generate=True)
t1 = thread_with_trace(target=model.generate, kwargs=generation_kwargs)
t1.start() 
response = ""
for new_text in streamer:
    response += new_text
    print(new_text, end="")
    res = match_pattern(response)
    if res is not None:
        t1.kill()
        t1.join()
        break
print(res)

SyntaxError: invalid syntax (3861758028.py, line 15)

In [ ]:
inputs = tokenizer([prompt2], return_tensors="pt").to('cuda')
streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=128, top_p=1, temperature=0.7, return_dict_in_generate=True)
t2 = thread_with_trace(target=model.generate, kwargs=generation_kwargs)
t2.start() 
response = ""
for new_text in streamer:
    response += new_text
    print(new_text, end="")
    res = match_pattern(response)
    if res is not None:
        t2.kill()
        t2.join()
        break

print(res)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 
    {
        "reasoning": "The user is responding positively to the greeting.",
        "prompt": "",
        "message": "That's great to hear! How was your week so far? Was there anything particularly exciting or interesting that happened to you?"
    } 

{'reasoning': 'The user is responding positively to the greeting.', 'message': "That's great to hear! How was your week so far? Was there anything particularly exciting or interesting that happened to you?", 'prompt': ''}
